In [ ]:
#加上 ! 執行 pip指令、安裝Hyperas。
!pip install hyperas


In [ ]:
from hyperopt import hp
from hyperopt import Trials, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
import numpy as np

def prepare_data():
    """
    準備資料
    """
    # 在此導入外部資料庫。
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import KFold
    from tensorflow.keras.utils import to_categorical
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Activation, Dropout

    # 將train.csv讀入pandas的DataFrame。
    train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
    train_x = train.drop(['label'], axis=1)  # 從train取出圖像資料
    train_y = train['label']               # 從train取出正確答案
    test_x = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

    # 將train資料分為訓練資料跟驗證資料。
    kf = KFold(n_splits=4, shuffle=True, random_state=123)
    tr_idx, va_idx = list(kf.split(train_x))[0]
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # 將圖像像素值除以255.0，限制在0 ~ 1.0範圍內，並轉換為numpy.array。
    tr_x, va_x = np.array(tr_x / 255.0), np.array(va_x / 255.0)

    # 將正確答案轉換為One-hot encoding呈現。
    tr_y = to_categorical(tr_y, 10)
    va_y = to_categorical(va_y, 10)

    return tr_x, tr_y, va_x, va_y

def create_model(tr_x, tr_y):
    """
    生成模型
    """
    # 生成Sequential object。
    model = Sequential()

    # 第1層神經元數量為784。
    model.add(Dense(784,
                    input_dim=tr_x.shape[1],
                    activation={{choice(['tanh', 'relu'])}}
                   ))
    # 在0.2 ~ 0.4的範圍內探尋第1層的丟棄率。
    model.add(Dropout({{quniform(0.2, 0.4, 0.05)}}))

    # 第2層神經元數量為200。
    model.add(Dense(200,
                    activation={{choice(['tanh', 'relu'])}}
                   ))
    # 在0.2 ~ 0.4的範圍內探尋第2層的丟棄率。
    model.add(Dropout({{quniform(0.2, 0.4, 0.05)}}))

    # 第3層神經元數量為25。
    model.add(Dense(25,
                    activation={{choice(['tanh', 'relu'])}}
                   ))
    # 在0.2 ~ 0.4的範圍內探尋第3層的丟棄率。
    model.add(Dropout({{quniform(0.2, 0.4, 0.05)}}))

    # 配置輸出層。激活函數使用Softmax。
    model.add(Dense(10, activation="softmax"))

    # 編譯模型。
    # 優化器定調為RMSprop。
    model.compile( loss="categorical_crossentropy",
                  optimizer='rmsprop',
                  metrics=["accuracy"])

    # 續練次數為20次。
    epoch = 20
    # 將小批次的尺寸嘗試設定為100與200。
    batch_size = {{choice([100, 200])}}
    result = model.fit( tr_x, tr_y,
                       epochs=epoch,
                       batch_size=batch_size,
                       validation_data=(va_x, va_y),
                       verbose=0)

    # 簡單地輸出訓練時的結果。
    validation_acc = np.amax(result.history['val_accuracy'])
    print('Best validation acc of epoch:', validation_acc)

    # 持續探尋以求得出validation_acc最小值。
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

# 執行探尋，次數為100。
best_run, best_model = optim.minimize(model=create_model,
                                      data=prepare_data,
                                      algo=tpe.suggest,
                                      max_evals=100,
                                      eval_space=True,
                                      notebook_name='__notebook_source__',
                                      trials=Trials())


In [ ]:
# 輸出準確率最好的模型。
print(best_model.summary())
# 輸出準確率最佳的參數值。
print(best_run)

# 使用驗證資料驗證模型。
_, _, va_x, va_y = prepare_data()
val_loss, val_acc = best_model.evaluate(va_x, va_y)
print("val_loss: ", val_loss) # 輸出損失。
print("val_acc: ", val_acc) # 輸出準確率。
